In [1]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix

In [2]:
df = pd.read_csv('dnd_monsters_simpler.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    float64
 2   type       401 non-null    object 
 3   size       401 non-null    object 
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      172 non-null    object 
 7   legendary  35 non-null     object 
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int64(2), object(5)
memory usage: 44.0+ KB


## Primeiro teste: Prevendo o CR (Challenge Rating) da criatura
### Convertendo os dados para numéricos (para que o GaussianNB funcione)
Fizemos exatamente igual ao KNN, transformando alguns campos em numéricos com Label Encoder, e deixando o Legendary como 1 ou 0.

In [3]:
df['legendary'] = df['legendary'].replace(['Legendary'], 1)
df['legendary'] = df['legendary'].fillna(0)

categorical_mappings = dict()
categorical_columns = ['type', 'size', 'speed']

for col in categorical_columns:
    le = LabelEncoder().fit(df[col])
    categorical_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    df[col] = le.transform(df[col])

print(categorical_mappings)

df.loc[df['cr'] == 0.125, 'cr'] = '1/8'
df.loc[df['cr'] == 0.25, 'cr'] = '1/4'
df.loc[df['cr'] == 0.5, 'cr'] = '1/2'

for i in range(len(df['cr'])):
    df['cr'][i] = str(df['cr'][i]).split('.')[0]

df['cr'].astype('object')

{'type': {'aberration': 0, 'beast': 1, 'celestial': 2, 'construct': 3, 'dragon': 4, 'elemental': 5, 'fey': 6, 'fiend': 7, 'giant': 8, 'humanoid': 9, 'monstrosity': 10, 'ooze': 11, 'plant': 12, 'swarm': 13, 'undead': 14}, 'size': {'Gargantuan': 0, 'Huge': 1, 'Large': 2, 'Medium': 3, 'Small': 4, 'Tiny': 5}, 'speed': {'fly': 0, 'fly, swim': 1, 'swim': 2, nan: 3}}


C:\Users\leona\AppData\Local\Temp\ipykernel_23184\2740291270.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cr'][i] = str(df['cr'][i]).split('.')[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_23184\2740291270.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cr'][i] = str(df['cr'][i]).split('.')[0]
C:\Users\leona\AppData\Local\Temp\ipykernel_23184\2740291270.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cr'

0      1/4
1       10
2      1/4
3       14
4       16
      ... 
396      3
397      4
398      1
399     26
400    1/4
Name: cr, Length: 401, dtype: object

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    object 
 2   type       401 non-null    int32  
 3   size       401 non-null    int32  
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      401 non-null    int32  
 7   legendary  401 non-null    float64
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int32(3), int64(2), object(2)
memory usage: 39.3+ KB


,name,cr,type,size,ac,hp,speed,legendary,str,dex,con,int,wis,cha
0,aarakocra,1/4,9,3,12,13,0,0.0,10.0,14.0,10.0,11.0,12.0,11.0
1,aboleth,10,0,2,17,135,2,1.0,21.0,9.0,15.0,18.0,15.0,18.0
2,acolyte,1/4,9,3,10,9,3,0.0,10.0,10.0,10.0,10.0,14.0,11.0
3,adult-black-dragon,14,4,1,19,195,1,1.0,23.0,14.0,21.0,14.0,13.0,17.0
4,adult-blue-dragon,16,4,1,19,225,0,1.0,25.0,10.0,23.0,16.0,15.0,19.0


## Preparando para a divisão de testes
Primeiro removemos a coluna name que não será usada para prever os dados

Segundo preparamos o X (sendo o dataframe sem o CR) e o Y (sendo a coluna CR)

Separamos em 70% de treino e 30% de teste

In [5]:
nameless_df = df.drop('name', axis=1)
X = nameless_df.drop('cr', axis=1)
y = nameless_df['cr']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

Podemos ver que o score de acurácia nem chegou a 25%, o que é pior que o KNN.

Novamente, a matriz de confusão é simplesmente uma loucura para ler, visto que são muitas dimensões.

In [9]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(acc)
print(cm)

0.2396694214876033
[[5 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 1 0 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 6 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 5 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 3 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0

## Segundo teste: Prevendo o CR (Challenge Rating) da criatura, porém filtrando para menos opções
Nesse segundo teste, nós tentamos diminuir o range de 0 a 30, para apenas 4 valores: Fácil, Médio, Difícil, Mortal, usando o seguinte critério para a conversão:

 - Fácil: 5 ou menos
 - Médio: 6 - 12
 - Difícil: 13 - 20
 - Mortal: 21 ou mais

### Convertendo os dados para numéricos (para que o GaussianNB funcione)
Igual da última vez, porém realizamos a conversão de CR

In [10]:
df = pd.read_csv('dnd_monsters_simpler.csv')

df['legendary'] = df['legendary'].replace(['Legendary'], 1)
df['legendary'] = df['legendary'].fillna(0)

categorical_mappings = dict()
categorical_columns = ['type', 'size', 'speed']

for col in categorical_columns:
    le = LabelEncoder().fit(df[col])
    categorical_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    df[col] = le.transform(df[col])

print(categorical_mappings)

def convert_cr(cr):
    if float(cr) <= 5:
        return 'Fácil'
    if 5 < float(cr) <= 12:
        return 'Médio'
    if 12 < float(cr) <= 20:
        return 'Difícil'
    if float(cr) > 20:
        return 'Mortal'

df['cr'] = df['cr'].apply(convert_cr)

df['cr'].astype('object')

{'type': {'aberration': 0, 'beast': 1, 'celestial': 2, 'construct': 3, 'dragon': 4, 'elemental': 5, 'fey': 6, 'fiend': 7, 'giant': 8, 'humanoid': 9, 'monstrosity': 10, 'ooze': 11, 'plant': 12, 'swarm': 13, 'undead': 14}, 'size': {'Gargantuan': 0, 'Huge': 1, 'Large': 2, 'Medium': 3, 'Small': 4, 'Tiny': 5}, 'speed': {'fly': 0, 'fly, swim': 1, 'swim': 2, nan: 3}}


0        Fácil
1        Médio
2        Fácil
3      Difícil
4      Difícil
        ...   
396      Fácil
397      Fácil
398      Fácil
399     Mortal
400      Fácil
Name: cr, Length: 401, dtype: object

In [11]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    object 
 2   type       401 non-null    int32  
 3   size       401 non-null    int32  
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      401 non-null    int32  
 7   legendary  401 non-null    float64
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int32(3), int64(2), object(2)
memory usage: 39.3+ KB


,name,cr,type,size,ac,hp,speed,legendary,str,dex,con,int,wis,cha
0,aarakocra,Fácil,9,3,12,13,0,0.0,10.0,14.0,10.0,11.0,12.0,11.0
1,aboleth,Médio,0,2,17,135,2,1.0,21.0,9.0,15.0,18.0,15.0,18.0
2,acolyte,Fácil,9,3,10,9,3,0.0,10.0,10.0,10.0,10.0,14.0,11.0
3,adult-black-dragon,Difícil,4,1,19,195,1,1.0,23.0,14.0,21.0,14.0,13.0,17.0
4,adult-blue-dragon,Difícil,4,1,19,225,0,1.0,25.0,10.0,23.0,16.0,15.0,19.0


## Preparando para a divisão de testes
Primeiro removemos a coluna name que não será usada para prever os dados

Segundo preparamos o X (sendo o dataframe sem o CR) e o Y (sendo a coluna CR)

Separamos em 70% de treino e 30% de teste

In [12]:
nameless_df = df.drop('name', axis=1)
X = nameless_df.drop('cr', axis=1)
y = nameless_df['cr']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

In [13]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(acc)
print(cm)

0.859504132231405
[[ 5  0  4  1]
 [ 1 72  0  7]
 [ 0  0  7  0]
 [ 0  4  0 20]]


Aqui a acurácia do GaussianNB ganha, por muito pouco, da do KNN, chegando em quase 86%.

Nossa interpretação sobre isso é: Visto que o Naive Bayes utiliza probabilidades de cada atributo, e no final adquire uma probabilidade para cada valor do campo a ser previsto: **Quanto mais categorias possíveis para a coluna de output do Naive Bayes, menor são as probabilidades em geral, e mais próximas elas se tornam. Assim o Naive Bayes acaba escolhendo a maior probabilidade, mesmo que ela ganhe apenas em 1% ou menos da segunda colocada, ou 2% da terceira. Os empates são mais frequentes**

A Matriz de confusão é bem mais legível dessa vez. A matriz está bem melhor em geral também.

Os erros ainda são presentes, e acreditamos que os fatores apontados no KNN (como a falta de outros atributos que também são bons indicativos de poder para a criatura) também se aplicam aqui.

## Terceiro teste: Prevendo se uma criatura é Lendária ou não
Vamos ao último teste. Assim como no KNN vamos fazer tudo que fizemos novamente, porém não mexeremos no CR e vamos usar o y como Legendary.

Colocaremos o Legendary como Yes e No assim como no KNN.

In [14]:
df = pd.read_csv('dnd_monsters_simpler.csv')

df['legendary'] = df['legendary'].replace(['Legendary'], 'Yes')
df['legendary'] = df['legendary'].fillna('No')

categorical_mappings = dict()
categorical_columns = ['type', 'size', 'speed']

for col in categorical_columns:
    le = LabelEncoder().fit(df[col])
    categorical_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    df[col] = le.transform(df[col])

print(categorical_mappings)

{'type': {'aberration': 0, 'beast': 1, 'celestial': 2, 'construct': 3, 'dragon': 4, 'elemental': 5, 'fey': 6, 'fiend': 7, 'giant': 8, 'humanoid': 9, 'monstrosity': 10, 'ooze': 11, 'plant': 12, 'swarm': 13, 'undead': 14}, 'size': {'Gargantuan': 0, 'Huge': 1, 'Large': 2, 'Medium': 3, 'Small': 4, 'Tiny': 5}, 'speed': {'fly': 0, 'fly, swim': 1, 'swim': 2, nan: 3}}


In [15]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       401 non-null    object 
 1   cr         401 non-null    float64
 2   type       401 non-null    int32  
 3   size       401 non-null    int32  
 4   ac         401 non-null    int64  
 5   hp         401 non-null    int64  
 6   speed      401 non-null    int32  
 7   legendary  401 non-null    object 
 8   str        401 non-null    float64
 9   dex        401 non-null    float64
 10  con        401 non-null    float64
 11  int        401 non-null    float64
 12  wis        401 non-null    float64
 13  cha        401 non-null    float64
dtypes: float64(7), int32(3), int64(2), object(2)
memory usage: 39.3+ KB


,name,cr,type,size,ac,hp,speed,legendary,str,dex,con,int,wis,cha
0,aarakocra,0.25,9,3,12,13,0,No,10.0,14.0,10.0,11.0,12.0,11.0
1,aboleth,10.00,0,2,17,135,2,Yes,21.0,9.0,15.0,18.0,15.0,18.0
2,acolyte,0.25,9,3,10,9,3,No,10.0,10.0,10.0,10.0,14.0,11.0
3,adult-black-dragon,14.00,4,1,19,195,1,Yes,23.0,14.0,21.0,14.0,13.0,17.0
4,adult-blue-dragon,16.00,4,1,19,225,0,Yes,25.0,10.0,23.0,16.0,15.0,19.0


In [16]:
nameless_df = df.drop('name', axis=1)
X = nameless_df.drop('legendary', axis=1)
y = nameless_df['legendary']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

Nos scores, pudemos usar o precision e o f1

In [17]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prc = precision_score(y_test, y_pred, average="binary", pos_label="No")
f1 = f1_score(y_test, y_pred, average="binary", pos_label="No")

cm = confusion_matrix(y_test, y_pred)
print(acc)
print(prc)
print(f1)
print(cm)

0.9090909090909091
0.98989898989899
0.9468599033816424
[[98 10]
 [ 1 12]]


Os scores em si estão muito melhores dessa vez, já que o atributo previsto é binário. Ficaram bem parecidos com o KNN, também, embora os do KNN tenham ficado levemente melhores, mas acreditamos que não seja o suficiente para escolhê-lo ao invés do Naive Bayes.

No KNN a matriz de confusão preveu 7 valores errôneos, aqui foram 11.